### To-do list
- [x] Test connection to the API
- [x] Read cache file
- [x] Create a function that will generate valid certification numbers
- [x] Create functions for all two API's
- [x] Organize all the card information he needs in a spreadsheet
- [x] Save photos on a different folder
- [x] Create links to the photos in the spreadsheet

### Notes
the beginning of the new lighthouse label started somewhere in the 274XXXXX range. from 200XXXX to 273XXXXX were the blank then hologram labels (2010 or so to 2017). since PSA used the 3XXXXXXX range years ago, they skipped it for now. then the 400XXXXX started up and is currently in the 480XXXXX range.

it started years ago numerically in order, not sure what happened with the skipping around. the certs beginning with 0, 1, 3 and 5 are all older numbers usually found on older 1990's-2000's holders. certs starting with 8's are usually PSA/DNA blue labels and now used on reds also for DNA authentic items.

Yes, cards that were graded as cert #20,000,000 were done in approximately mid-2012.

Around 23,000,000 they started adding the hologram to the front of flips. Around 28,000,000 they started adding the PSA Lighthouse label.
Higher than 30,000,000, they would need to have the PSA Lighthouse label. Many of PSA's VERY OLD grades had Cert numbers from 30Million, 40M, 90M, so they had to skip those numbers when their numbering system finally caught up to those certs. So if you see any cards between 30M and 43M, they need to have the most current lighthouse label to be graded between 2012 and now. It's still slightly possible they were graded 20 years ago and were sent in for reholdering recently, but if they're from your personal collection, that doesn't really apply because you would know which cards those are.

### Conclusions
- Do not use cert numbers from 30M to 43M
- Cert numbers started in the 20M range and are currently in the 48M range

In [50]:
import requests
import pandas as pd
import random
import os

In [1]:
api_token = 'RDH-9Ozp1H9i-rGhHtZ48DcpI_lr8einCkIklByDbGuUbeIothnq_DiEp0Axz517rNHupElOguFOFmpp_1Xc37B1fHh7f7vy9ubZStKqoen3R_NgfZO-AjegRGTXD11rU9x18Q3MccDrzmMNBkceOb8YQfP3R9pzd3_GQ46zSthuCzhzrWSUi0ejSOLeh1NOaQ8M0-ply7hjJyfQTXYzVUmWWBS8bkz01r1cpOLG1QN8wpMb6yhNgSEh6VtAwgzurcMAwm1puxpxn1-awYOfId63uWBktY9sHorH6YzQIe1hWeXI'

### Declaring functions

In [52]:
### API Function
def get_card_info(cert_number, access_token):
    card_info = f'https://api.psacard.com/publicapi/cert/GetByCertNumber/{cert_number}'

    headers = {
        "Authorization": f"bearer {access_token}"
    }

    response = requests.get(card_info, headers=headers)

    if response.status_code == 200:
        # API call was successful
        data = response.json()
        return data
    else:
        # Handle API error
        print(f"Error: {response.status_code}")
        return None
    
def get_card_images(cert_number, access_token):
    images = f'https://api.psacard.com/publicapi/cert/GetImagesByCertNumber/{cert_number}'

    headers = {
        "Authorization": f"bearer {access_token}"
    }

    response = requests.get(images, headers=headers)

    if response.status_code == 200:
        # API call was successful
        data = response.json()
        return data
    else:
        # Handle API error
        print(f"Error: {response.status_code}")
        return None
    
def download_image(url, folder_path, filename):
    # Create the folder if it doesn't exist
    os.makedirs(folder_path, exist_ok=True)

    # Get the image content from the URL
    response = requests.get(url)
    if response.status_code == 200:
        # Save the image to the specified folder with the given filename
        with open(os.path.join(folder_path, filename), "wb") as f:
            f.write(response.content)
        print(f"Image downloaded: {filename}")
    else:
        print(f"Failed to download image: {filename}")
    
### Cert Number Generator
def generate_certification_number(consulted_numbers):
    while True:
        if not consulted_numbers:  # If the set is empty, generate a random number and return it
            return random.randint(60000000, 87000000)
        
        cert_number = random.randint(60000000, 87000000)
        if cert_number not in consulted_numbers:
            return cert_number
        
### Read and update cache file
def load_cache(cache_file):
    consulted_numbers = set()
    if os.path.exists(cache_file):
        with open(cache_file, "r") as f:
            for line in f:
                consulted_numbers.add(int(line.strip()))
    return consulted_numbers

def update_cache(cache_file, consulted_numbers):
    with open(cache_file, "w") as f:
        for cert_number in consulted_numbers:
            f.write(str(cert_number) + "\n")

### Reading cache file

In [ ]:
card_info_list = []
cache_file = "consulted_cert_numbers.txt"

# Load consulted certification numbers from the cache file
consulted_numbers = load_cache(cache_file)

# Reading database
database_path = "Card_database.xlsx"

# Check if the existing file exists
if os.path.exists(database_path):
    # Load the existing Excel file into a DataFrame
    existing_df = pd.read_excel(database_path)
else:
    # If the file doesn't exist, create an empty DataFrame
    existing_df = pd.DataFrame()

### Loop to get info for each of the 400 cards

In [ ]:
while len(card_info_list) < 400:
    # Generates the cert number
    cert_number = generate_certification_number(consulted_numbers)
    try:
        # Calls the API to get card info
        card_info = get_card_info(cert_number, api_token)

        if not card_info:
            consulted_numbers.add(cert_number)
            continue  # Skip this card if data doesn't exist
        
        cert_number = card_info['PSACert']['CertNumber']
        card_year = card_info['PSACert']['Year']
        card_brand = card_info['PSACert']['Brand']
        card_number = card_info['PSACert']['CardNumber']
        card_player = card_info['PSACert']['Subject']
        card_variety = card_info['PSACert']['Variety']
        card_grade = card_info['PSACert']['CardGrade']

        card_images = get_card_images(cert_number, api_token)

        try:
            # URL of the image to download
            back_image = card_images[0]['ImageURL']
            front_image = card_images[1]['ImageURL']

            # Folder path to save the image
            folder_path = 'images'

            # Filename for the image
            filename_back = f'{cert_number}_back.jpg'
            filename_front = f'{cert_number}_front.jpg'

            download_image(back_image, folder_path, filename_back)
            download_image(front_image, folder_path, filename_front)

            back_hyperlink = f'=HYPERLINK("{filename_back}", "Image")'
            front_hyperlink = f'=HYPERLINK("{filename_front}", "Image")'

        except:
            image_hyperlink = 'No Image Available'

        # Add card information to the list
        card_info_list.append({
            'Cert number': cert_number,
            'Year': card_year,
            'Brand': card_brand,
            'Card number': card_number,
            'Player': card_player,
            'Variety': card_variety,
            'Grade': card_grade,
            'ImageLink': image_hyperlink  # Use the hyperlink formula
        })

        consulted_numbers.add(cert_number)

    except:
        consulted_numbers.add(cert_number)
    

### Updating cache and saving cert numbers to an existing Excel file

In [51]:
update_cache(cache_file, consulted_numbers)
new_df = pd.DataFrame(card_info_list)

# Concatenate existing and new DataFrames
final_df = pd.concat([existing_df, new_df], ignore_index=True)

# Save the combined DataFrame to the Excel file
final_df.to_excel(database_path, index=False)

NameError: name 'cache_file' is not defined

# Test cells

In [6]:
def get_card_info(cert_number, access_token):
    url = f"https://api.psacard.com/publicapi/cert/GetByCertNumber/{cert_number}"
    headers = {
        "Authorization": f"bearer {access_token}"
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        # API call was successful
        data = response.json()
        return data
    else:
        # Handle API error
        print(f"Error: {response.status_code}")
        return None

# Example usage
cert_number = "16992626"  # Example PSA certification number

card_info = get_card_info(cert_number, api_token)
if card_info:
    print(card_info)
else:
    print("Failed to retrieve card information.")

{'PSACert': {'CertNumber': '16992626', 'SpecID': 184351, 'SpecNumber': '0101633000', 'LabelType': 'LighthouseLabel', 'ReverseBarCode': True, 'Year': '1963', 'Brand': 'TOPPS', 'Category': 'BASEBALL CARDS', 'CardNumber': '300', 'Subject': 'WILLIE MAYS', 'Variety': '', 'IsPSADNA': False, 'IsDualCert': False, 'GradeDescription': 'GEM MT 10', 'CardGrade': 'GEM MT 10', 'TotalPopulation': 1, 'TotalPopulationWithQualifier': 0, 'PopulationHigher': 0}}


In [46]:
cert_number = "16992626"  # Example PSA certification number

card_info = get_card_images(cert_number, api_token)
if card_info:
    print(card_info)
else:
    print("Failed to retrieve card information.")

[{'IsFrontImage': False, 'ImageURL': 'https://d1htnxwo4o0jhw.cloudfront.net/cert/138754979/Oo_EP8WE40-ylkFv3D78IA.jpg'}, {'IsFrontImage': True, 'ImageURL': 'https://d1htnxwo4o0jhw.cloudfront.net/cert/138754979/oh-FZb2dIUm1td51eC0HVA.jpg'}]


In [48]:
card_info[0]['ImageURL']

'https://d1htnxwo4o0jhw.cloudfront.net/cert/138754979/Oo_EP8WE40-ylkFv3D78IA.jpg'

In [ ]:
columns = ['Cert Number',
           'Year',
           'Brand',
           'Card number',
           'Player', # Subject
           'Variety',
           'Grade',
           'Front image',
           'Back image'
           ]

In [11]:
cert_number = card_info['PSACert']['CertNumber']
card_year = card_info['PSACert']['Year']
card_brand = card_info['PSACert']['Brand']
card_number = card_info['PSACert']['CardNumber']
card_player = card_info['PSACert']['Subject']
card_variety = card_info['PSACert']['Variety']
card_grade = card_info['PSACert']['CardGrade']

{'CertNumber': '16992626',
 'SpecID': 184351,
 'SpecNumber': '0101633000',
 'LabelType': 'LighthouseLabel',
 'ReverseBarCode': True,
 'Year': '1963',
 'Brand': 'TOPPS',
 'Category': 'BASEBALL CARDS',
 'CardNumber': '300',
 'Subject': 'WILLIE MAYS',
 'Variety': '',
 'IsPSADNA': False,
 'IsDualCert': False,
 'GradeDescription': 'GEM MT 10',
 'CardGrade': 'GEM MT 10',
 'TotalPopulation': 1,
 'TotalPopulationWithQualifier': 0,
 'PopulationHigher': 0}

In [43]:
import random

def generate_certification_number(consulted_numbers):
    while True:
        if not consulted_numbers:  # If the set is empty, generate a random number and return it
            return random.randint(60000000, 87000000)
        
        if cert_number not in consulted_numbers:
            return random.randint(60000000, 87000000)
        
generate_certification_number([])

73168831